## Предсказание стоимости жилья

Нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

### Описание данных

    longitude — широта;
    latitude — долгота;
    housing_median_age — медианный возраст жителей жилого массива;
    total_rooms — общее количество комнат в домах жилого массива;
    total_bedrooms — общее количество спален в домах жилого массива;
    population — количество человек, которые проживают в жилом массиве;
    households — количество домовладений в жилом массиве;
    median_income — медианный доход жителей жилого массива;
    median_house_value — медианная стоимость дома в жилом массиве;
    ocean_proximity — близость к океану.

In [1]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
RANDOM_SEED = 1

In [2]:
pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator
        




In [3]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California house - Logistic regression") \
                    .getOrCreate()



In [4]:

df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)

# Подготовка данных

In [5]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [6]:
df.show(6)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Найти и посчитать пропуски

In [7]:
df2 = df.select([F.count(F.when(F.col(c).contains('None') | \
                            F.col(c).contains('NULL') | \
                            (F.col(c) == '' ) | \
                            F.col(c).isNull() | \
                            F.isnan(c), c 
                           )).alias(c)
                    for c in df.columns])

print('Пропуски обнаружены в следующих данных:')
display(df2.toPandas())

Пропуски обнаружены в следующих данных:


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,0,0,0,207,0,0,0,0,0


In [8]:
display(df.describe().toPandas()) # и тут видно, что в колонке total_bedrooms есть пробелы

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Заполнить пропуски

In [9]:
total_bedrooms_median = int(df.stat.approxQuantile('total_bedrooms',[0.5],0)[0])

In [10]:
df = df.na.fill(total_bedrooms_median,['total_bedrooms'])

In [11]:
display(df.describe().toPandas()) 

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,536.8388565891473,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.39187792168866,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [12]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms',\
                   'population', 'households', 'median_income']
target = 'median_house_value' 

Разделить выборки на тестовую и обучающую.

In [13]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

16507 4133


Закодировать OHE категориальный признак.

In [14]:
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols]) 
model_ind = indexer.fit(train_data) # переведёт в числовые значения
train_data = model_ind.transform(train_data)
cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3) 

+---------------+
|ocean_proximity|
+---------------+
|     NEAR OCEAN|
|     NEAR OCEAN|
|     NEAR OCEAN|
+---------------+
only showing top 3 rows



In [15]:
test_data = model_ind.transform(test_data)
test_data.select(cols).show(3) 

+---------------+
|ocean_proximity|
+---------------+
|     NEAR OCEAN|
|     NEAR OCEAN|
|     NEAR OCEAN|
+---------------+
only showing top 3 rows



In [16]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])
model_ohe = encoder.fit(train_data)
train_data = model_ohe.transform(train_data) 

In [17]:
cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
train_data.select(cols).show(3) 

+---------------+
|ocean_proximity|
+---------------+
|     NEAR OCEAN|
|     NEAR OCEAN|
|     NEAR OCEAN|
+---------------+
only showing top 3 rows



In [18]:
test_data = model_ohe.transform(test_data)
cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
test_data.select(cols).show(3) 

+---------------+
|ocean_proximity|
+---------------+
|     NEAR OCEAN|
|     NEAR OCEAN|
|     NEAR OCEAN|
+---------------+
only showing top 3 rows



In [19]:
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
train_data = categorical_assembler.transform(train_data) 
test_data = categorical_assembler.transform(test_data)

Шкалировать числовые, что сделает данные более однородными.

In [20]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
train_data = numerical_assembler.transform(train_data) 
test_data = numerical_assembler.transform(test_data) 

In [21]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")
model_scal = standardScaler.fit(train_data)
train_data = model_scal.transform(train_data)
test_data = model_scal.transform(test_data)

Собрать категории и числа в один вектор.

In [22]:
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)
train_data.select(all_features).show(6) 

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-62.197108857510...|
|       (4,[2],[1.0])|     [-62.172099967740...|
|       (4,[2],[1.0])|     [-62.172099967740...|
|       (4,[2],[1.0])|     [-62.157094633879...|
|       (4,[2],[1.0])|     [-62.147091077971...|
|       (4,[2],[1.0])|     [-62.137087522063...|
+--------------------+-------------------------+
only showing top 6 rows



In [23]:
test_data.select(all_features).show(6) 

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-62.152092855925...|
|       (4,[2],[1.0])|     [-62.117080410247...|
|       (4,[2],[1.0])|     [-62.117080410247...|
|       (4,[2],[1.0])|     [-62.117080410247...|
|       (4,[2],[1.0])|     [-62.112078632293...|
|       (4,[2],[1.0])|     [-62.107076854339...|
+--------------------+-------------------------+
only showing top 6 rows



# Обучение моделей

In [23]:
rating_list = {
    'RMSE': [],
    'MAE': [],
    'R2': []
}

In [24]:
def linear_modelling(train_data, test_data, target, featuresCol, model_name, rating_list):
    
    lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol=target, featuresCol=featuresCol)
    model = lr.fit(train_data)
    predictions = model.transform(test_data)
    predictedLabes = predictions.select("median_house_value", "prediction")
    print(f'\nПредсказания в модели {model_name}\n')
    predictedLabes.show() 
    
    # RMSE
    evaluator = RegressionEvaluator(
    labelCol=target, predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    print('RMSE: ', rmse)
    
    
    # MAE
    evaluator = RegressionEvaluator(
    labelCol=target, predictionCol="prediction", metricName="mae")
    mae = evaluator.evaluate(predictions)
    print('MAE: ', mae)
    
    # R2
    evaluator = RegressionEvaluator(
    labelCol=target, predictionCol="prediction", metricName="r2")
    r2 = evaluator.evaluate(predictions)
    print('R2: ', r2) 
    
    rating_list['RMSE'].append(rmse)
    rating_list['MAE'].append(mae)
    rating_list['R2'].append(r2)
    
    return rating_list
    
    

###### Модель с категориальными данными

In [25]:
rating_list = linear_modelling(train_data, test_data, target, "features", 'with categorical features', rating_list)

22/12/23 16:01:47 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/12/23 16:01:47 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS



Предсказания в модели with categorical features

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          111400.0|205809.43412493146|
|           90100.0|213761.66524101468|
|           69000.0| 192581.8750578689|
|           70000.0| 164664.8557414678|
|          107000.0|202656.34720332548|
|           70500.0|180048.00687120575|
|           75500.0|175439.51443297835|
|          105900.0| 163352.3898437852|
|           92700.0|201711.38674109755|
|           74700.0|185271.20161318267|
|          100600.0|206968.60554956715|
|           86400.0|209809.90555929695|
|          103100.0| 68151.04637840902|
|          100500.0|179644.31556450785|
|           86900.0|168858.95467298594|
|           83000.0| 194572.3416313387|
|           55000.0| 221500.4903474555|
|           72300.0|163522.53220209433|
|           92600.0|187036.42389814183|
|           97300.0| 178042.5164423762|
+------------------+----------

###### Модель без категориальных данных



In [26]:
rating_list = linear_modelling(train_data, test_data, target,"numerical_features_scaled", 'without categorical features', rating_list)


Предсказания в модели without categorical features

+------------------+-------------------+
|median_house_value|         prediction|
+------------------+-------------------+
|          111400.0| 168845.08940031705|
|           90100.0|   167891.363062134|
|           69000.0|  146722.7882020059|
|           70000.0| 116392.49507634761|
|          107000.0| 158683.67926313402|
|           70500.0| 134862.60277372925|
|           75500.0| 131755.20177613152|
|          105900.0|  89188.68539767852|
|           92700.0|  158787.9798915293|
|           74700.0| 141972.39029269386|
|          100600.0| 158192.40289747296|
|           86400.0|  163346.0313168415|
|          103100.0|-15593.851544664241|
|          100500.0| 129359.44031561399|
|           86900.0|  118848.1475385041|
|           83000.0|  144601.6522390009|
|           55000.0| 193895.29076012317|
|           72300.0| 127731.15228190925|
|           92600.0|  133821.0854512765|
|           97300.0|  139729.6056876951|
+---

# Анализ результатов

In [27]:
rating_table = {
    'Model': ['with categorical features', 'without categorical features'],
    'RMSE': rating_list['RMSE'],
    'MAE': rating_list['MAE'],
    'R2': rating_list['R2']
}
rating_table = pd.DataFrame(rating_table)
display(rating_table)

,Model,RMSE,MAE,R2
0,with categorical features,68995.562196,50004.940088,0.64871
1,without categorical features,69870.890782,51115.015112,0.63974


RMSE - Это корень от квадрата ошибки MSE, которая измеряет среднюю сумму квадратной разности между фактическим значением и прогнозируемым значением для всех точек данных. 

MAE - метрика измеряет среднюю сумму абсолютной разницы между фактическим значением и прогнозируемым значением.

R² - это соотношение между тем, насколько хороша наша модель, и тем, насколько хороша модель наивного среднего. 
По этой метрике обе модели средненькие, с категориальными признаками немного лучше.

По другим же метрикам у нас довольно странные предсказания - как и в примерах, различия между целевым признаком и предсказанием очень большие. 

